#installs

In [ ]:
! pip install fastapi kaleido python-multipart uvicorn numpy==1.24.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that

In [ ]:
! pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.3/313.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 104.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 127.0 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.8.0.76
    Uninstalling opencv-python-4.8.0.76:
      Successfully uninstalled opencv-python-4.8.0.76
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.6.0
    Unins

#imports

In [4]:
import torch
import albumentations as alb
from albumentations.pytorch import ToTensorV2
import torch.nn as nn

In [5]:
from PIL import Image
import os
from torch.utils.data import Dataset
import numpy as np

In [6]:
import multiprocessing

#Configuration

In [26]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

2

In [32]:


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TRAIN_DIR = "/content/drive/MyDrive/CopyMoveForgeryDetection/fold0"
VAL_DIR = "/content/drive/MyDrive/CopyMoveForgeryDetection/validate"
BATCH_SIZE = 1
LEARNING_RATE = 1e-5
LAMBDA_IDENTITY = 0.0
LAMBDA_CYCLE = 10
NUM_WORKERS = 2
NUM_EPOCHS = 10
LOAD_MODEL = False
SAVE_MODEL = True
CHECKPOINT_GEN_H = "genX.pth.tar"
CHECKPOINT_GEN_Z = "genY.pth.tar"
CHECKPOINT_CRITIC_H = "criticX.pth.tar"
CHECKPOINT_CRITIC_Z = "criticY.pth.tar"


### Transformations before feeding
transforms = alb.Compose(
    [
        alb.Resize(width=512, height=512),
        alb.HorizontalFlip(p=0.5),
        alb.Normalize(mean=[0.5, 0.5, 0.5], std=[
                      0.5, 0.5, 0.5], max_pixel_value=255),
        ToTensorV2(),
    ],
    additional_targets={"image0": "image"}, is_check_shapes=False
)


# Dataset Pre-process

In [10]:
class HorseZebraDataset(Dataset):
    def __init__(self, root_y, root_x, transform=None):
        self.root_y = root_y
        self.root_x = root_x
        self.transform = transform

        self.y_images = os.listdir(root_y)
        self.x_images = os.listdir(root_x)
        self.length_dataset = max(len(self.y_images), len(self.x_images)) # 1000, 1500
        self.y_len = len(self.y_images)
        self.x_len = len(self.x_images)

    def __len__(self):
        return self.length_dataset

    def __getitem__(self, index):
        y_img = self.y_images[index % self.y_len]
        x_img = self.x_images[index % self.x_len]

        zebra_path = os.path.join(self.root_y, y_img)
        horse_path = os.path.join(self.root_x, x_img)

        y_img = np.array(Image.open(zebra_path).convert("RGB"))
        x_img = np.array(Image.open(horse_path).convert("RGB"))

        if self.transform:
            augmentations = self.transform(image=y_img, image0=x_img)
            y_img = augmentations["image"]
            x_img = augmentations["image0"]

        return y_img, x_img






# Architecture
<img src="https://imgs.search.brave.com/ycwQQQ4PY2s_Ai8M_pGgFYCJckpOr46RCAz7A8vBWqo/rs:fit:860:0:0/g:ce/aHR0cHM6Ly93d3cu/c2NhbGVyLmNvbS90/b3BpY3MvaW1hZ2Vz/L2FyY2hpdGVjdHVy/ZS1pbi1jeWNsZWdh/bi53ZWJw">

# Generator
<img src="https://imgs.search.brave.com/5BWq-72d0cEZSySvkUyo5IcA0OmdyT0ijrMjXmx7aJ0/rs:fit:860:0:0/g:ce/aHR0cHM6Ly93d3cu/c2NhbGVyLmNvbS90/b3BpY3MvaW1hZ2Vz/L2dlbmVyYXRvci1h/cmNoaXRlY3R1cmUt/aW4tY3ljbGVnYW4u/d2VicA">

In [11]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, use_act=True, **kwargs):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, padding_mode="reflect", **kwargs) if down else nn.ConvTranspose2d(in_channels, out_channels, **kwargs),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True) if use_act else nn.Identity(),
        )

    def forward(self, x):
        return self.conv(x)


###
class DoubleConv(nn. Module):
    def __init__(self, in_channels, out_channels):
        super (DoubleConv, self).__init__()
        self.conv = nn.Sequential(
        nn.Conv2d (in_channels, out_channels, 3, 1, 1, bias=False),
        nn. BatchNorm2d (out_channels),
        nn. ReLU(inplace=True),
        nn. Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
        nn. BatchNorm2d (out_channels),
        nn. ReLU(inplace=True),
        )
    def forward (self, x):
        return self.conv(x)



In [12]:
class UNET(nn.Module):
    """
    UNET Module
    """
    def __init__(self, in_channels=3, out_channels=3, features=[64, 128, 256, 512]):
        super (UNET, self).__init__()
        self.upsample_steps = nn.ModuleList()
        self.downsample_steps = nn.ModuleList()
        self.pool = nn. MaxPool2d (kernel_size=2, stride=2)

        # DownsamplingBlock of UNET
        for feature in features:
            self.downsample_steps.append (DoubleConv(in_channels, feature))
            in_channels = feature

        # ##
        self.bottleneck = DoubleConv(features[-1], features [-1]*2)

        # UpsamplingBlock of UNET
        for feature in reversed(features):
            self.upsample_steps.append(nn.ConvTranspose2d(feature*2, feature, kernel_size=2, stride=2))

            self.upsample_steps.append(DoubleConv(feature*2, feature))

        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        # downsample
        for downsample in self.downsample_steps:
            x = downsample(x)
            skip_connections.append(x)
            x = self.pool(x)

        # bottleneck - before upsample
        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        # upsample
        for index in range(0, len(self.upsample_steps), 2):
            x = self.upsample_steps[index](x)
            skip_connection = skip_connections[index//2]
            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.upsample_steps[index+1](concat_skip)
        return self.final_conv(x)

In [13]:
###
class Generator(nn.Module):
    def __init__(self, img_channels, num_features=64):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                img_channels,
                num_features,
                kernel_size=7,
                stride=1,
                padding=3,
                padding_mode="reflect",
            ),
            nn.InstanceNorm2d(num_features),
            nn.ReLU(inplace=True),
        )
        self.down_blocks = nn.ModuleList(
            [
                ConvBlock(
                    num_features, num_features * 2, kernel_size=3, stride=2, padding=1
                ),
                ConvBlock(
                    num_features * 2,
                    num_features * 4,
                    kernel_size=3,
                    stride=2,
                    padding=1,
                ),
            ]
        )


        # UNET START
        self.UNET_block = UNET(in_channels=36, out_channels=256, features=[64, 128, 256])


        # UNET END

        self.up_blocks = nn.ModuleList(
            [
                ConvBlock(
                    num_features * 4,
                    num_features * 2,
                    down=False,
                    kernel_size=3,
                    stride=2,
                    padding=1,
                    output_padding=1,
                ),
                ConvBlock(
                    num_features * 2,
                    num_features * 1,
                    down=False,
                    kernel_size=3,
                    stride=2,
                    padding=1,
                    output_padding=1,
                ),
            ]
        )

        self.last = nn.Conv2d(
            num_features * 1,
            img_channels,
            kernel_size=7,
            stride=1,
            padding=3,
            padding_mode="reflect",
        )

    def forward(self, x):
        print(x.shape)
        x = self.initial(x)
        for layer in self.down_blocks:
            x = layer(x)
            print(x.shape)
        x = self.UNET_block(x)
        print(x.shape)
        for layer in self.up_blocks:
            x = layer(x)
            print(x.shape)
        return torch.tanh(self.last(x))

In [14]:
def test():
    img_channels = 3
    img_size = 256
    x = torch.randn((2, img_channels, img_size, img_size))
    gen = UNET(img_channels)
    print(gen(x).shape)

In [15]:
test()

torch.Size([2, 3, 256, 256])


# Discriminator

<img src="https://media.geeksforgeeks.org/wp-content/uploads/20200605220731/Discriminator.jpg">

In [16]:
class Block(nn.Module):
    """
    Basic building block for the discriminator called Block.
    This block consists of a convolutional layer with instance normalization and leaky ReLU activation.
    """
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                4,
                stride,
                1,
                bias=True,
                padding_mode="reflect",
            ),
            nn.InstanceNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=True),
        )

    ### Will perform Forward Prop
    def forward(self, x):
        return self.conv(x)





class Discriminator(nn.Module):
    """
    Intregated Discriminator Network
    """
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                in_channels,
                features[0],
                kernel_size=4,
                stride=2,
                padding=1,
                padding_mode="reflect",
            ),
            nn.LeakyReLU(0.2, inplace=True),
        )

        ### Layers for Discriminator
        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(
                Block(in_channels, feature,
                      stride=1 if feature == features[-1] else 2)
            )
            in_channels = feature

        ### Layer before output
        layers.append(
            nn.Conv2d(
                in_channels,
                1,
                kernel_size=4,
                stride=1,
                padding=1,
                padding_mode="reflect",
            )
        )
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.initial(x)
        return torch.sigmoid(self.model(x))


In [17]:
### Test Discriminator
def test():
    x = torch.randn((5, 3, 256, 256))
    model = Discriminator(in_channels=3)
    preds = model(x)
    print(preds.shape)

In [18]:
test()

torch.Size([5, 1, 30, 30])


# Utilities

In [19]:
import os, numpy as np
import copy

In [20]:
def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


def seed_everything(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# Training

In [28]:
import sys
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
from torchvision.utils import save_image

In [29]:
def train_fn(disc_H, disc_Z, gen_Z, gen_H, loader, opt_disc, opt_gen, l1, mse, d_scaler, g_scaler):
    H_reals = 0
    H_fakes = 0
    loop = tqdm(loader, leave=True)

    for idx, (zebra, horse) in enumerate(loop):
        zebra = zebra.to(DEVICE)
        horse = horse.to(DEVICE)

        # Train Discriminators H and Z
        with torch.cuda.amp.autocast():
            fake_horse = gen_H(zebra)
            D_H_real = disc_H(horse)
            D_H_fake = disc_H(fake_horse.detach())
            H_reals += D_H_real.mean().item()
            H_fakes += D_H_fake.mean().item()
            D_H_real_loss = mse(D_H_real, torch.ones_like(D_H_real))
            D_H_fake_loss = mse(D_H_fake, torch.zeros_like(D_H_fake))
            D_H_loss = D_H_real_loss + D_H_fake_loss

            fake_zebra = gen_Z(horse)
            D_Z_real = disc_Z(zebra)
            D_Z_fake = disc_Z(fake_zebra.detach())
            D_Z_real_loss = mse(D_Z_real, torch.ones_like(D_Z_real))
            D_Z_fake_loss = mse(D_Z_fake, torch.zeros_like(D_Z_fake))
            D_Z_loss = D_Z_real_loss + D_Z_fake_loss

            # put it togethor
            D_loss = (D_H_loss + D_Z_loss) / 2

        opt_disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train Generators H and Z
        with torch.cuda.amp.autocast():
            # adversarial loss for both generators
            D_H_fake = disc_H(fake_horse)
            D_Z_fake = disc_Z(fake_zebra)
            loss_G_H = mse(D_H_fake, torch.ones_like(D_H_fake))
            loss_G_Z = mse(D_Z_fake, torch.ones_like(D_Z_fake))

            # cycle loss
            cycle_zebra = gen_Z(fake_horse)
            cycle_horse = gen_H(fake_zebra)
            cycle_zebra_loss = l1(zebra, cycle_zebra)
            cycle_horse_loss = l1(horse, cycle_horse)

            # identity loss (remove these for efficiency if you set lambda_identity=0)
            identity_zebra = gen_Z(zebra)
            identity_horse = gen_H(horse)
            identity_zebra_loss = l1(zebra, identity_zebra)
            identity_horse_loss = l1(horse, identity_horse)

            # add all togethor
            G_loss = (
                loss_G_Z
                + loss_G_H
                + cycle_zebra_loss * LAMBDA_CYCLE
                + cycle_horse_loss * LAMBDA_CYCLE
                + identity_horse_loss * LAMBDA_IDENTITY
                + identity_zebra_loss * LAMBDA_IDENTITY
            )

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

        if idx % 200 == 0:
            save_image(fake_horse * 0.5 + 0.5, f"horse_{idx}.png")
            save_image(fake_zebra * 0.5 + 0.5, f"zebra_{idx}.png")

        loop.set_postfix(H_real=H_reals / (idx + 1),
                         H_fake=H_fakes / (idx + 1))




In [30]:
def main():
    disc_H = Discriminator(in_channels=3).to(DEVICE)
    disc_Z = Discriminator(in_channels=3).to(DEVICE)
    gen_Z = UNET(in_channels=3).to(DEVICE)
    gen_H = UNET(in_channels=3).to(DEVICE)
    opt_disc = optim.Adam(
        list(disc_H.parameters()) + list(disc_Z.parameters()),
        lr=LEARNING_RATE,
        betas=(0.5, 0.999),
    )

    opt_gen = optim.Adam(
        list(gen_Z.parameters()) + list(gen_H.parameters()),
        lr=LEARNING_RATE,
        betas=(0.5, 0.999),
    )

    L1 = nn.L1Loss()
    mse = nn.MSELoss()

    if LOAD_MODEL:
        load_checkpoint(
            CHECKPOINT_GEN_H,
            gen_H,
            opt_gen,
            LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_GEN_Z,
            gen_Z,
            opt_gen,
            LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_CRITIC_H,
            disc_H,
            opt_disc,
            LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_CRITIC_Z,
            disc_Z,
            opt_disc,
            LEARNING_RATE,
        )

    dataset = HorseZebraDataset(
        root_x=TRAIN_DIR + "/forged",
        root_y=TRAIN_DIR + "/mask",
        transform=transforms,
    )
    val_dataset = HorseZebraDataset(
        root_x=VAL_DIR + "/forged",
        root_y=VAL_DIR + "/mask",
        transform=transforms,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        pin_memory=True,
    )
    loader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True,
    )
    g_scaler = torch.cuda.amp.GradScaler(enabled=False)
    d_scaler = torch.cuda.amp.GradScaler(enabled=False)

    for epoch in range(NUM_EPOCHS):
        train_fn(disc_H, disc_Z, gen_Z, gen_H, loader, opt_disc, opt_gen, L1, mse, d_scaler, g_scaler)

        if SAVE_MODEL:
            save_checkpoint(gen_H, opt_gen, filename=CHECKPOINT_GEN_H)
            save_checkpoint(gen_Z, opt_gen, filename=CHECKPOINT_GEN_Z)
            save_checkpoint(disc_H, opt_disc,
                            filename=CHECKPOINT_CRITIC_H)
            save_checkpoint(disc_Z, opt_disc,
                            filename=CHECKPOINT_CRITIC_Z)




In [33]:
main()

100%|██████████| 150/150 [01:33<00:00,  1.60it/s, H_fake=0.424, H_real=0.74]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 150/150 [01:33<00:00,  1.60it/s, H_fake=0.331, H_real=0.845]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 150/150 [01:33<00:00,  1.60it/s, H_fake=0.301, H_real=0.874]


...

=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 150/150 [01:34<00:00,  1.59it/s, H_fake=0.308, H_real=0.858]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 150/150 [01:34<00:00,  1.60it/s, H_fake=0.297, H_real=0.868]


=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


In [34]:
test()

torch.Size([5, 1, 30, 30])


In [42]:
optimizer = optim.Adam(
        list(gen_Z.parameters()) + list(gen_H.parameters()),
        lr=LEARNING_RATE,
        betas=(0.5, 0.999),
    )

NameError: ignored

In [ ]:
from torch.optim import optimizer

In [37]:
load_checkpoint('genX.pth.tar')

TypeError: ignored

In [41]:
# Example of loading the model
checkpoint = torch.load('genX.pth.tar')
model = UNET  # Initialize your model class
model.load_state_dict(checkpoint['state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']


TypeError: ignored

In [40]:
checkpoint['state_dict'].keys()

odict_keys(['upsample_steps.0.weight', 'upsample_steps.0.bias', 'upsample_steps.1.conv.0.weight', 'upsample_steps.1.conv.1.weight', 'upsample_steps.1.conv.1.bias', 'upsample_steps.1.conv.1.running_mean', 'upsample_steps.1.conv.1.running_var', 'upsample_steps.1.conv.1.num_batches_tracked', 'upsample_steps.1.conv.3.weight', 'upsample_steps.1.conv.4.weight', 'upsample_steps.1.conv.4.bias', 'upsample_steps.1.conv.4.running_mean', 'upsample_steps.1.conv.4.running_var', 'upsample_steps.1.conv.4.num_batches_tracked', 'upsample_steps.2.weight', 'upsample_steps.2.bias', 'upsample_steps.3.conv.0.weight', 'upsample_steps.3.conv.1.weight', 'upsample_steps.3.conv.1.bias', 'upsample_steps.3.conv.1.running_mean', 'upsample_steps.3.conv.1.running_var', 'upsample_steps.3.conv.1.num_batches_tracked', 'upsample_steps.3.conv.3.weight', 'upsample_steps.3.conv.4.weight', 'upsample_steps.3.conv.4.bias', 'upsample_steps.3.conv.4.running_mean', 'upsample_steps.3.conv.4.running_var', 'upsample_steps.3.conv.4.nu